In [256]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.stats import zscore


In [257]:
#reading rheast and extracting cnames
rheas = pd.read_csv(r'data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')


In [258]:
# rename cname column to district
rheas.rename(columns={'cname':'district'}, inplace=True)

In [259]:
# convert columns to lower and cname names
rheas.columns=rheas.columns.str.lower()
rheas.district=rheas.district.str.lower()
rheas.district=rheas.district.str.replace('-', '', regex=True)
rheas.district=rheas.district.str.replace(' ', '', regex=True)

In [260]:
# clean rheas survey data
print(len(rheas))
#drop duplicate inputs
rheas = rheas.drop_duplicates()
print(len(rheas))
rheas

295779
295779


,id,gid,district,season,ccode,ensemble,planting,harvest,wsgd,gwad
0,1083083,94,nkeyema,1998 1,110011,29,1999-01-01,1999-05-18,0.000,3716
1,1083087,94,nkeyema,1998 1,110011,33,1999-01-01,1999-05-18,0.000,3716
2,1056596,83,mushindano,1998 1,108011,15,1999-01-01,1999-05-18,0.000,3716
3,1085429,95,nsama,1998 1,107009,15,1999-01-01,1999-05-18,0.000,3716
4,929310,30,kalabo,1998 1,110001,41,1999-01-01,1999-05-18,0.000,3716
...,...,...,...,...,...,...,...,...,...,...
295774,1289442,63,mambwe,2022 1,103005,8,2023-01-30,2023-06-04,0.000,4090
295775,1229437,38,kasama,2022 1,107003,3,2023-01-30,2023-06-07,0.182,4447
295776,1229441,38,kasama,2022 1,107003,7,2023-01-30,2023-06-06,0.000,4161
295777,1289479,63,mambwe,2022 1,103005,45,2023-01-30,2023-06-04,0.000,4230


In [261]:
# drop unnecessary columns
rheas.rename(columns={'cname':'district'}, inplace=True)
rheas=rheas.drop(['id','gid', 'ccode', 'ensemble'], axis=1)

In [262]:
# # convert appropriate columns to numeric
# rheas.planted_ha=pd.to_numeric(rheas.planted_ha)
# rheas.harvested_ha=pd.to_numeric(rheas.harvested_ha)
# rheas.production_mt=pd.to_numeric(rheas.production_mt)
# rheas['yield(mt/ha)']=pd.to_numeric(rheas['yield(mt/ha)'])
# convert dates and calculate yield
rheas['planting']=pd.to_datetime(rheas['planting'], format="mixed")
rheas['year'] = rheas['planting'].dt.year
rheas['yield']=rheas['gwad']/1000
rheas['month'].unique()
rheas

KeyError: 'month'

In [ ]:
def rheas_to_season(data):
    df=[]
    season_list = [2, 2, 2, 2,2, 2, 0, 0, 0, 1, 1, 1]
    month_to_season=dict(zip(range(1,13), season_list ))
    data['season2']= data['planting'].dt.month.map(month_to_season)
    # Extract data only for the seasons
    part1= data.loc[data['season2']==1]
    part2= data.loc[data['season2']==2]
    part3=pd.concat([part1, part2], axis=0, ignore_index=True)
    rheas_part1= part3.loc[part3['season2']==1]
    rheas_part2= part3.loc[part3['season2']==2]
    part4=pd.concat([rheas_part1, rheas_part2], axis=0, ignore_index=True)
    for year in list(part4.year.unique()):
       df1=rheas_part1.loc[rheas_part1['year']==year]
       df2=rheas_part2.loc[rheas_part2['year']==year+1]
       df3=pd.concat([df1, df2], axis=0, ignore_index=True)
       df3['season_year']=year
       df.append(df3)
       df4=pd.concat(df,axis=0,ignore_index=True)
       df5=df4.groupby(['district','season_year'],as_index=False).agg({'yield':'mean'})
    return df5

rheas=rheas_to_season(data=rheas)
rheas

,district,season_year,yield
0,chadiza,1999,4.261113
1,chadiza,2000,4.168920
2,chadiza,2001,4.147630
3,chadiza,2002,4.040450
4,chadiza,2003,4.237460
...,...,...,...
2779,zimba,2018,3.656320
2780,zimba,2019,3.926810
2781,zimba,2020,4.151760
2782,zimba,2021,3.899850


In [ ]:
#calculate z scores of yield and flag any anomalies
def map_zscore(data):
    df = []
    for d in list(data.district.unique()):
        df1 = data.loc[data['district']==d].copy()
        df1['zyield']=zscore(df1['yield'])
        df.append(df1)
        df2=pd.concat(df,axis=0, ignore_index=True)
        # add flag
        df2['flag'] = 0
        df2['flag'].loc[(df2['zyield']>1.96)|(df2['zyield']<-1.96)]= 1
        df2['flag'].loc[(df2['zyield']>2.58)|((df2['zyield']<-2.58))] = 2
    return df2
rheas1=map_zscore(rheas)
rheas1

C:\Users\kasit\AppData\Local\Temp\ipykernel_10840\2228904251.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['flag'].loc[(df2['zyield']>1.96)|(df2['zyield']<-1.96)]= 1
C:\Users\kasit\AppData\Local\Temp\ipykernel_10840\2228904251.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['flag'].loc[(df2['zyield']>2.58)|((df2['zyield']<-2.58))] = 2
C:\Users\kasit\AppData\Local\Temp\ipykernel_10840\2228904251.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,district,season_year,yield,zyield,flag
0,chadiza,1999,4.261113,1.502019,0
1,chadiza,2000,4.168920,0.965961,0
2,chadiza,2001,4.147630,0.842171,0
3,chadiza,2002,4.040450,0.218973,0
4,chadiza,2003,4.237460,1.364487,0
...,...,...,...,...,...
2779,zimba,2018,3.656320,-1.990595,1
2780,zimba,2019,3.926810,-0.441294,0
2781,zimba,2020,4.151760,0.847165,0
2782,zimba,2021,3.899850,-0.595715,0


In [ ]:
# save to csv
rheas.to_csv('data/rheas/zambia_rheas_district_data_clean.csv',index=False)
rheas

,district,season_year,yield,zyield,flag
0,chadiza,1999,4.261113,1.502019,0
1,chadiza,2000,4.168920,0.965961,0
2,chadiza,2001,4.147630,0.842171,0
3,chadiza,2002,4.040450,0.218973,0
4,chadiza,2003,4.237460,1.364487,0
...,...,...,...,...,...
2779,zimba,2018,3.656320,-1.990595,1
2780,zimba,2019,3.926810,-0.441294,0
2781,zimba,2020,4.151760,0.847165,0
2782,zimba,2021,3.899850,-0.595715,0


In [263]:
# read out shapefile and rename columns appropriately
# read data
zambia_district_shp=gpd.read_file('data/shp/zmb_district_cleaned.shp')

# rename preharvest column to same name as shapefile
rheas1.rename(columns={'district':'district_b','season_year':'year'}, inplace=True)

# merge
rheas_shp=pd.merge(zambia_district_shp,rheas1, on='district_b')
rheas1

,district_b,year,yield
0,chadiza,1999,4.261113
1,chadiza,2000,4.168920
2,chadiza,2001,4.147630
3,chadiza,2002,4.040450
4,chadiza,2003,4.237460
...,...,...,...
2779,zimba,2018,3.656320
2780,zimba,2019,3.926810
2781,zimba,2020,4.151760
2782,zimba,2021,3.899850


In [254]:
# aggregate to province level
rheas_province=rheas_shp.groupby(['province_b','year'], 
                                           as_index=False).agg({'yield': 'mean'})

In [255]:
rheas_province.to_csv('data/rheas/zambia_rheas_province_data_clean.csv', index=False)
rheas_province

,province_b,year,yield
0,central,1999,4.252927
1,central,2000,4.243914
2,central,2001,4.199693
3,central,2002,4.177999
4,central,2003,3.958135
...,...,...,...
245,western,2019,3.762460
246,western,2020,3.960419
247,western,2021,4.115611
248,western,2022,3.777532
